In [ ]:
# %load_ext nb_black
%load_ext autoreload
%autoreload 2

## Initialization

### Imports

In [ ]:
import os
import sys
import cv2
import json
import glob
import torch
import tifffile
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px

from tqdm.notebook import tqdm
from collections import Counter
from matplotlib import pyplot as plt
from scipy.ndimage import gaussian_filter

sys.path.append("../code/")

In [ ]:
from params import *
from utils.rle import *

from data.dataset import load_image

from utils.metrics import dice_scores_img
from utils.plots import plot_heatmap_preds, plot_contours_preds

### Load

In [ ]:
df_info = pd.read_csv(DATA_PATH + f"HuBMAP-20-dataset_information.csv")

### Data

In [ ]:
DATA_PATH

In [ ]:
root = TIFF_PATH
rle_path = DATA_PATH + "train_2_fix.csv"
reduce_factor = 2
df_mask = pd.read_csv(rle_path)

## Train viz

In [ ]:
THRESHOLD = 0.5

In [ ]:
mask_name = "afa5e8098" 

In [ ]:
img = load_image(os.path.join(TIFF_PATH_2, mask_name + ".tiff"), full_size=False, reduce_factor=2)

In [ ]:
rle = df_mask[df_mask['id'] == mask_name]['encoding']
mask = enc2mask(rle, (img.shape[1], img.shape[0]))

In [ ]:
plt.figure(figsize=(15, 15))
plt.imshow(img[3000: 5000, 3000: 5000])
plt.show()

## Extra data

In [ ]:
df_extra = pd.read_csv('../input/train_extra.csv').set_index('id')
DATA_PATH_EXTRA = DATA_PATH + "extra_tiff/"

In [ ]:
name = "SAS_21908_001"

In [ ]:
rle = df_extra.loc[name]

In [ ]:
img = tifffile.imread(DATA_PATH_EXTRA + name + ".tiff").squeeze()

In [ ]:
mask = enc2mask(rle, img.shape[:2][::-1])

In [ ]:
img.shape

In [ ]:
plt.figure(figsize=(15, 15))
plt.imshow(img[8000: 10000, 1000: 3000])
plt.show()

## New extra data

In [ ]:
ZENODO_PATH = "../input/zenodo/"
FACTOR = 2
ANNOT_PATH = DATA_PATH + "annotation_v3/"

In [ ]:
imgs = [p[:-4] for p in os.listdir(ZENODO_PATH) if p.endswith(".png") and "mask" not in p]

In [ ]:
SAVE = True

out_dir = "../input/zenodo_2/"
if not os.path.exists(out_dir):
    os.mkdir(out_dir)

In [ ]:
for img_name in tqdm(imgs):
    print(f' -> {img_name}')
    
    img = cv2.imread(ZENODO_PATH + img_name + ".png")
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    mask = cv2.imread(ZENODO_PATH + img_name + "_mask.png")    
    

    if img_name + ".json" in os.listdir(ANNOT_PATH):        
        mask_add_fc = np.zeros(mask.shape[:2], dtype=np.uint8)
        mask_add_g = np.zeros(mask.shape[:2], dtype=np.uint8)
        annot = json.load(open(ANNOT_PATH + img_name + ".json", 'r'))
        added = 0
        
        for info in annot:
            try:
                label = info['properties']['classification']['name']
            except:
                print('Label missing')  # was a FC
                label = "FC"
            poly = info['geometry']['coordinates']
            
            if label == "FC":
                try:
                    mask_add_fc = cv2.fillPoly(mask_add_fc, np.int32([poly]), True)
                except ValueError:
                    poly = np.concatenate([np.array(poly[i]).squeeze() for i in range(len(poly))])
                    mask_add_fc = cv2.fillPoly(mask_add_fc, np.int32([poly]), True)
            else:
                try:
                    mask_add_g = cv2.fillPoly(mask_add_g, np.int32([poly]), True)
                except ValueError:
                    poly = np.concatenate([np.array(poly[i]).squeeze() for i in range(len(poly))])
                    mask_add_g = cv2.fillPoly(mask_add_g, np.int32([poly]), True)
            

            added += 1
            
#         plt.figure(figsize=(15, 15))
#         plt.imshow(mask_add_g)
#         plt.show()

        mask[:, :, 1] += mask_add_g * 255
        mask[:, :, 2] += mask_add_fc * 255
    
        print(f"Added {added} glomerulis")
    
    img = cv2.resize(
        img,
        (img.shape[1] // FACTOR, img.shape[0] // FACTOR),
        interpolation=cv2.INTER_AREA,
    )
    mask = cv2.resize(
        mask,
        (mask.shape[1] // FACTOR, mask.shape[0] // FACTOR),
        interpolation=cv2.INTER_NEAREST,
    )
    
    mask = (mask[:, :, 1] // 255 + mask[:, :, 2] // 255 * 2).astype(np.uint8)
    
    if SAVE:
        cv2.imwrite(out_dir + img_name + ".png", img)
        np.save(out_dir + img_name + "_mask.npy", mask)
    
    contours_preds, _ = cv2.findContours(mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    cv2.polylines(img, contours_preds, True, (0.0, 255, 0.0), 2)
    
    plt.figure(figsize=(15, 15))
    plt.imshow(img)
    plt.show()

#     break
    

In [ ]:
ZENODO_PATH = "../input/zenodo_2/"
imgs = [p[:-4] for p in os.listdir(ZENODO_PATH) if p.endswith(".png") and "mask" not in p]

In [ ]:
for img_name in tqdm(imgs):
    print(f' -> {img_name}')
    
    img = cv2.imread(ZENODO_PATH + img_name + ".png")
    mask = np.load(ZENODO_PATH + img_name + "_mask.npy")

    contours_preds, _ = cv2.findContours((mask == 1).astype(np.uint8), cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    cv2.polylines(img, contours_preds, True, (0.0, 255, 0.0), 2)
    contours_preds, _ = cv2.findContours((mask == 2).astype(np.uint8), cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    cv2.polylines(img, contours_preds, True, (0.0, 0,255), 2)
    
    
    plt.figure(figsize=(15, 15))
    plt.imshow(img)
    plt.show()
    
#     break